# Loading the data

In [2]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("Datasets/news_processed_spacy.csv")
data.head()

,text,polarity,n_tokens,total_n_ents,n_org_ents,n_per_ents,n_gpe_ents,n_norp_ents,n_date_ents,entities,org_ents,per_ents,gpe_ents,norp_ents,date_ents
0,Kerry to go to Paris in gesture of sympathyU.S...,True,495,58,2,17,23,6,10,"['Kerry', 'Paris', 'State', 'John F. Kerry', '...","['State', 'the American Embassy']","['Kerry', 'John F. Kerry', 'Kerry', 'Laurent F...","['Paris', 'Paris', 'Paris', 'France', 'Sofia',...","['American', 'French', 'Israeli', 'European', ...","['Monday', 'later this week', 'Sunday', 'Thurs..."
1,The Battle of New York: Why This Primary Matte...,True,405,42,12,14,9,4,3,"['New York', 'Hillary Clinton', 'Donald Trump'...","['Trump', 'the White House', 'Trump', 'Sanders...","['Hillary Clinton', 'Donald Trump', 'Ted Cruz'...","['New York', 'Ohio', 'New York', 'New York', '...","['Republican', 'Republican', 'Democratic', 'In...","['year', 'this weekend', 'November']"
2,‘Britain’s Schindler’ Dies at 106A Czech stock...,True,148,24,4,4,8,4,4,"['Britain', 'Schindler’ Dies', 'Czech', 'Jewis...","['Schindler’ Dies', 'Winton', 'Winton', 'Winton']","['Dubbed “Britain’s Schindler', 'Nicholas Wint...","['Britain', 'Nazi Germany', 'Prague', 'Germany...","['Czech', 'Jewish', 'Jewish', 'German']","['the age of 106', 'March 1939', '2003', 'near..."
3,Fact check: Trump and Clinton at the 'commande...,True,2861,298,103,87,60,12,36,"['Trump', 'Clinton', 'Clinton', 'Donald Trump'...","['Trump', 'NBC', 'Trump', 'Obama', 'Trump', 't...","['Clinton', 'Clinton', 'Donald Trump', '• Clin...","['Iraq', 'Iraq', 'Obama', 'China', 'Saudi Arab...","['Republicans', 'Democratic', 'Republicans', '...","['Sept. 7', 'Today', 'Sept. 11, 2002', 'about ..."
4,Iran reportedly makes new push for uranium con...,True,813,77,12,7,39,4,15,"['Iran', 'U.S.', 'Iran', 'the final days', 'Th...","['The New York Times', 'Times', 'The Associate...","['Olli Heinonen', 'John Kerry', 'Edward Kenned...","['Iran', 'U.S.', 'Iran', 'Tehran', 'Russia', '...","['Western', 'French', 'German', 'Iranian']","['the final days', 'late Sunday', 'just two da..."


In [4]:
data_clean = data [["text", "polarity"]]
data_clean.head()

,text,polarity
0,Kerry to go to Paris in gesture of sympathyU.S...,True
1,The Battle of New York: Why This Primary Matte...,True
2,‘Britain’s Schindler’ Dies at 106A Czech stock...,True
3,Fact check: Trump and Clinton at the 'commande...,True
4,Iran reportedly makes new push for uranium con...,True


In [5]:
data_clean.replace({"True": 1, "Fake":0}, inplace=True)

<ipython-input-5-a9b801638842>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_clean.replace({"True": 1, "Fake":0}, inplace=True)
<ipython-input-5-a9b801638842>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean.replace({"True": 1, "Fake":0}, inplace=True)


In [6]:
data_clean["polarity"].value_counts()

,count
polarity,
1,3171
0,3164


In [7]:
data_clean.shape

(6335, 2)

In [8]:
data_clean.rename({"polarity": "labels"}, axis=1, inplace=True)

<ipython-input-8-7869ce53ec97>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean.rename({"polarity": "labels"}, axis=1, inplace=True)


## Splitting the Data

In [9]:
def random_indexes(number, percentage, df, used_indexes=False, used_indexes_2=False):

    """Manually sampling the indexes of the dataframe taking into account the previously sampled indexes

    """
    possible_indexes = list(df.index)

    if used_indexes:
        possible_indexes = [index for index in possible_indexes if index not in used_indexes]
    if used_indexes_2:
        possible_indexes = [index for index in possible_indexes if index not in used_indexes_2]

    df_sampleable = df[df.index.isin(possible_indexes)]

    n_samples = int(percentage*number)
    df_sample = df_sampleable.sample(n_samples)

    return df_sample

training_portion = data_clean.copy()
testing_portion = data_clean.copy()
validation_portion = data_clean.copy()

def is_double(list_1, list_2, list_3):
    same_index = list(set(list_1).intersection(set(list_2), set(list_3)))
    if len(same_index) != 0:
        return True
    else:
        return False

while is_double(list(training_portion.index), list(testing_portion.index), list(validation_portion.index)) == True:
    training_portion = random_indexes(6335, 0.7, training_portion)
    testing_portion = random_indexes(6335, 0.15, testing_portion, used_indexes = list(training_portion.index))
    validation_portion = random_indexes(6335, 0.15, validation_portion, used_indexes = list(training_portion.index), used_indexes_2 = list(testing_portion.index))


In [10]:
same_index = list(set(training_portion.index).intersection(set(testing_portion.index), set(validation_portion.index)))
same_index

[]

In [11]:
training_indexes = list(training_portion.index)
testing_indexes = list(testing_portion.index)
validation_indexes = list(validation_portion.index)

In [33]:
dict_indexes_training = {"training": training_indexes,
                         "testing": testing_indexes,
                         "validation": validation_indexes}

In [1]:
import json

with open("Datasets/data_split_indexes.json", "w") as f:
  json.dump(dict_indexes_training, f)


NameError: name 'dict_indexes_training' is not defined

In [12]:
training_portion["labels"].value_counts()

,count
labels,
0,2226
1,2208


In [13]:
validation_portion["labels"].value_counts()

,count
labels,
1,492
0,458


In [14]:
testing_portion["labels"].value_counts()

,count
labels,
0,480
1,470
